# Import data

In [ ]:
import pandas as pd
import numpy as np
import scipy as sci
import matplotlib as plt

df = pd.read_csv("Walmart_sales.csv")
df.head()

# Data Cleaning 

## Converting Date column into date time


In [ ]:
df["Date"] = pd.to_datetime(df['Date'], format="%d-%m-%Y")

print(df.dtypes)
df.head()

# Data Exploration

## Individual Feature Exploration

### Stores

#### Taking a look at the number of stores

In [ ]:
numStores = len(df["Store"].unique())
print("Number of Stores: " + str(numStores))

Clearly, we have a pretty large number of stores (around 10% of the stores in the US according to some quick research). 

#### Taking a look at the number of holidays per store 

In [ ]:
print(df[df["Holiday_Flag"] == 1].groupby("Store")["Holiday_Flag"].count().hist())

It looks like each store has exactly 10 holidays.

#### Taking a look at the average sales per store

In [ ]:

# Average sales per store differs 
df.groupby("Store")["Weekly_Sales"].mean().hist()

It seems like the average sales per store varies.

#### Taking a look at average temperature per store 

In [ ]:
df.groupby("Store")["Temperature"].mean().hist()

The distribution of average temperature per store is bimodal with a peak around 52 degrees and another peak around 70 degrees. 

It seems like the stores are in different regions and/or that the dates for each store span different time frames. 

Spoiler alert: when you take a look at average CPI per store it becomes clear that there is a temporal difference between stores. However, the geographic difference cannot be asserted. 

#### Taking a look at the average CPI per store 

In [ ]:
df.groupby("Store")["CPI"].mean().hist()

This supports the assertion that the stores come from different moments in time because CPI is fixed for each day. Thus, if the stores were all overlapping in time we'd see the same average for every store.

#### Taking a look at the average unemployment rate per store

In [ ]:
df.groupby("Store")["Unemployment"].mean().hist()

This once again supports the assertion that the data from these stores don't come from the same period of time. 

However, it is weird that the average unemployment rate per store and the avg CPI per store don't follow a similar distribution. This might be worth exploring.

#### Summary of findings after exploring the Store column


##### Observed
- Stores aren't aligned in time. That is, the dates of their sales data differ. 
- Average sales per store differ. Different levels of performance. 
- Every store has the same number of holidays.
- There are 45 stores.

##### Questions
- Are the stores in different regions?
- Why is the distribution of average unemployment rates different from the distribution of average CPI? 
    - Is unemployment rate local or national? 



### Date

In [ ]:
df.groupby("Date")["Date"].count().hist()
numUnique = len(df["Date"].unique())
print("The number of unique dates are: " + str(numUnique))

So we have 143 unique dates w/ each date having 45 corresponding instances.

In [ ]:
df.groupby(df["Date"].dt.month)["Date"].count().plot(kind="bar")

We do not have the same amount of data for each month of the year. So this again, supports the assertion that the data for each store is not set in the same time frame.

Now, lets take a look at the amount of data we have for each year.

In [ ]:
df.groupby(df["Date"].dt.year)["Date"].count().plot(kind="bar")

We have the most data on sales during 2011, then sales in 2010, and last sales in 2012.

### Weekly Sales

In [ ]:
df["Weekly_Sales"].hist()
mean = df["Weekly_Sales"].mean()
stdDev = df["Weekly_Sales"].std()

print("Mean: " + str(mean))
print("Std Dev: " + str(stdDev))

This is a really weird distribution... why is the distribution this way? Does this have to do with how the sales data was collected, with the way walmart operates it's stores, or some other factor we cannot think of?

### Holiday Flag

In [ ]:
import matplotlib.pyplot as plt

# Define the bin edges to align with tick marks
bins = [-0.25, 0.25, 0.75, 1.25]

non_holiday = df[df['Holiday_Flag'] == 0]
holiday = df[df['Holiday_Flag'] == 1]

plt.hist(non_holiday['Holiday_Flag'], bins=bins, color='blue', label='Non-Holiday', alpha=0.5)
plt.hist(holiday['Holiday_Flag'], bins=bins, color='green', label='Holiday', alpha=0.5)

plt.xlabel('Holiday Flag')
plt.ylabel('Frequency')
plt.title('Distribution of Holiday Flag')
plt.legend()

# Customizing x-axis ticks and labels
plt.xticks([0, 0.5, 1], ['Non-Holiday', '', 'Holiday'])

plt.show()

From this analysis, we can infer that there is a very low percentage of weeks that are holidays. Consequently, holiday weeks, defined as 1, are expected to have a smaller effect on sales compared to holiday weeks. As, their lower holiday weeks vs. vs. holiday weeks in general, will be taking a look at the relationship between holidays and sales further below for stotres 1 - 4.

### Tempature

In [ ]:
print(df["Temperature"].describe())
df["Temperature"].hist()

There is a large variation in temperatures. This could mean one of three things: 
- The stores are in different geographic locations. 
- All the stores are not in different geograhic locations but rather they are all located in a region that sees both temperature extremes a year.
- **Climate change** 

### Fuel Price

In [ ]:
# View histogram for fuel price
df['Fuel_Price'].hist()

We have a bimodal distribution for our fuel prices.

In [ ]:
# View summary statistics
mean = df['Fuel_Price'].mean()
stdDev = df['Fuel_Price'].std()

print(f"Mean: {mean}")
print(f"Standard Devation: {stdDev}")


In today's dollars adjusted for inflation that would be a mean gas price of about $4.70. This unusually high average may affect the generalizability of our ML model / conclusions

### Consumer Price Index

In [ ]:
# View histogram
df['CPI'].hist()

It looks like we have a really weird discrete set of values. I wonder why there seems to be a chunk missing. From my understanding CPI is a continuous index that doesn't simply jump out like that. 

To dig deeper into why this occured let's look at CPI for each store (it should be roughly the same because the data is from the same time frame but it help determine the cause).  

In [ ]:
# Histogram for store 1
df[df['Store'] == 1]['CPI'].hist()

In [ ]:
frame = df[df['Store'] == 1]['CPI']
print(f"Range: [{frame.min()}, {frame.max()}]")

It looks store 1 is continuous in the rough range [210, 223].

In [ ]:
# View histogram of cpi for store 2
df[df['Store'] == 2]['CPI'].hist()

In [ ]:
frame = df[df['Store'] == 2]['CPI']
print(f"Range: [{frame.min()}, {frame.max()}]")


Store 2's CPI values seem to be continuous in a very similar range as store 1.

In [ ]:
# View histogram of CPI's for store 3
df[df['Store'] == 3]['CPI'].hist()

In [ ]:
frame = df[df['Store'] == 3]['CPI']
print(f"Range: [{frame.min()}, {frame.max()}]")

Again, very similar range of CPI values. So, stores 1, 2, and 3 all have similar CPI ranges and store 4 seems to be an outlier in the CPI data.

In [ ]:
# View histogram of CPI values for store 4
df[df['Store'] == 4]['CPI'].hist()

In [ ]:
frame = df[df['Store'] == 4]['CPI']
print(f"Range: [{frame.min()}, {frame.max()}]")

As suspected, store 4 is completely misasligned with the other stores in terms of it's CPI values. Hence, why we had that huge gap in data in the histogram of ALL CPI values. 

### Unemployment

In [ ]:
df['Unemployment'].hist()

print("Avg unemployment rate: " + str(df["Unemployment"].mean()))

It looks like we have relatively high unemployment rates throughout most of our data. This is an important attribute to note as it may affect the generalizability of our ML Model and/or conclusions.

## Feature vs Sales

### Tempature vs Sales

We want to use a hypothesis test to figure out whether temperature affects sales.

*Null Hypothesis*: The average temperature of the area has no effect on the sales.

*Alternative Hypothesis*: The average temperature of the area does have an effect on the sales.

Assume that we have a significance level of 0.05.

First, let's plot the stores, with their respective average temperature and weekly sales. We'll use a scatter plot, with each dot being a store from the set.

In [ ]:
store_avgs = df.groupby("Store").mean("Temperature")

temp = store_avgs["Temperature"]
sales = store_avgs["Weekly_Sales"]

ts = np.vstack([temp,sales])
z = sci.stats.gaussian_kde(ts)(ts)

fig, plot = plt.subplots()

plt.title("Temperature vs. Weekly Sales")
plt.xlabel("Average Temperature")
plt.ylabel("Weekly Sales (in million USD)")

plot.scatter(temp, sales, c=z, s=100)

plt.show()

We can see that the stores are scattered around pretty evenly, with small clusters around the average temperature of 45-55 and 65-75 degree range.

Let's check the correlation between the two variables.

In [ ]:
result = sci.stats.pearsonr(store_avgs["Temperature"], store_avgs["Weekly_Sales"], alternative="two-sided")

print(result)

Notice that the pearson correlation coefficient is -0.076, meaning that it is slightly negative. This means that as the average temperature increases, then the weekly sales decrease. Since the p value is larger than our alpha value (0.617), we fail to reject the null hypothesis. In other words, we can see that the average temperature does not have an effect on weekly sales.

### Fuel Price vs Sales

We can also use a hypothesis test to check if fuel prices have an effect on sales.

*Null Hypothesis: Fuel Prices do not have any effect on weekly sales.*

*Alternative Hypothesis: Fuel prices do have an effect on weekly sales.*

Assume the significance level as 0.05.

Let's first create a scatter plot graphing the correlation between fuel price and weekly sales.

In [ ]:
store_avgs = df.groupby("Store").mean("Fuel_Price")

temp = store_avgs["Fuel_Price"]
sales = store_avgs["Weekly_Sales"]

ts = np.vstack([temp,sales])
z = sci.stats.gaussian_kde(ts)(ts)

fig, plot = plt.subplots()

plt.title("Fuel Price vs. Weekly Sales")
plt.xlabel("Average Fuel Price")
plt.ylabel("Weekly Sales (in million USD)")

plot.scatter(temp, sales, c=z, s=100)

plt.show()

The fuel prices seem divided into three categories of prices, ranging from 3.20, 3.45, and 3.6 USD per gallon. With those three subdivisions, we can see that the weekly sales seem evenly spread out. From inspection, there looks like there's a very, VERY slight downard trend when the fuel price goes up.

Let's run a pearson correlation coefficient test to find out the correlation:

In [ ]:
result = sci.stats.pearsonr(store_avgs["Fuel_Price"], store_avgs["Weekly_Sales"], alternative="two-sided")

print(result)

Notice that the correlation coefficient is 0.06, which is slightly positive. As fuel prices go up, so do weekly sales. However, this is a very, very small increase, to the point where it isn't significant enough.

Since our p value is 0.65, which is way higher than our alpha value of 0.05, we fail to reject the null hypothesis. In other words, fuel price does not have an effect on weekly sales.

### CPI vs Sales

We want to use a hypothesis test to figure out whether cpi affects sales.

*Null Hypothesis*: The average cpi does no effect on the sales.

*Alternative Hypothesis*: The average cpi does have an effect on the sales.

Assume that we have a significance level of 0.05.

First, let's plot the stores, with their respective average cpi and weekly sales. We'll use a scatter plot, with each dot being a store from the set.

In [ ]:
store_avgs = df.groupby("Store").mean("CPI")

temp = store_avgs["CPI"]
sales = store_avgs["Weekly_Sales"]

ts = np.vstack([temp,sales])
z = sci.stats.gaussian_kde(ts)(ts)

fig, plot = plt.subplots()

plt.title("CPI vs. Weekly Sales")
plt.xlabel("CPI Temperature")
plt.ylabel("Weekly Sales (in million USD)")

plot.scatter(temp, sales, c=z, s=100)

plt.show()

We observe that the stores are divided into two groups: some stores cluster around 140 CPI on the left-hand side, while others cluster around 200 CPI on the right-hand side. Interestingly, there appears to be an even distribution of revenue among these points, indicating comparable sales performance across stores despite differences in CPI.

In [ ]:
result = sci.stats.pearsonr(store_avgs["CPI"], store_avgs["Weekly_Sales"], alternative="two-sided")

print(result)

Notice that the Pearson correlation coefficient is -0.0765, indicating a slight negative correlation. This suggests that as the average CPI increases, weekly sales decrease. However, since the p-value is larger than our alpha value (0.617), we fail to reject the null hypothesis. In other words, we can conclude that the average CPI does not have a significant effect on weekly sales. This conclusion is supported by the scatter plot visualization, where no noticeable change in the relationship between average CPI and weekly sales is observed as CPI increases or decreases.

### Unemployment vs Sales

We want to now test if unemployment rates have an effect on the stores' average weekly sales.

*Null Hypothesis:* The unemployment rate has no effect on the stores' average weekly sales.

*Alternative Hypothesis:* The unemployment rate does have an effect on the stores' average weekly sales.

Assume that our significance level is 0.05.

We can graph a scatter plot of each store:

In [ ]:
temp = store_avgs["Unemployment"]
sales = store_avgs["Weekly_Sales"]

ts = np.vstack([temp,sales])
z = sci.stats.gaussian_kde(ts)(ts)

fig, plot = plt.subplots()

plt.title("Unemployment vs. Weekly Sales")
plt.xlabel("Average Unemployment Rate")
plt.ylabel("Weekly Sales (in million USD)")

plot.scatter(temp, sales, c=z, s=100)

plt.show()

We can see that most of the stores' average unemployment rates are between 6-9 percent, with sales being scattered pretty evenly across the board.

Let's now check the correlation between the two variables:

In [ ]:
result = sci.stats.pearsonr(store_avgs["Unemployment"], store_avgs["Weekly_Sales"], alternative="two-sided")

print(result)

Notice that the correlation coefficient is -0.112, which is slightly negative. This means that as unemployment rates go up, the weekly sales will go slighly down. Since our p value is 0.46, which is greater than 0.05, we fail to reject the null hypothesis. In other words, the unemployment rate does not have an effect on weekly sales.

### Holiday vs Sales

Will be taking a look at the relationship between holidays and sales for stores 1 through 4, as we know from the Holiday flag, there is lower frequency of holiday weeks then non-holiday weeks.

In [ ]:
import matplotlib.pyplot as plt

# Store 3 with non-Holiday's
non_holiday_1 = df[(df['Store'] == 1) & (df['Holiday_Flag'] == 0)].iloc[:140]

# Store 4 with holidays
holiday_1 = df[(df['Store'] == 1) & (df['Holiday_Flag'] == 1)].iloc[:140]


# Plot sales data for Store 1 during non-holiday weeks
plt.plot(non_holiday_1.index, non_holiday_1['Weekly_Sales'], color='blue', label='Non-Holiday Weeks')

# Plot sales data for Store 1 during holiday weeks
plt.plot(holiday_1.index, holiday_1['Weekly_Sales'], color='red', label='Holiday Weeks')

plt.xlabel('Week')
plt.ylabel('Weekly Sales (In Millions)')
plt.title('Weekly Sales for Store 1')
plt.legend()
plt.grid(True)  
plt.tight_layout()
plt.show()

The interaction between the sales data during non-holiday and holiday weeks highlights the impact of holidays on consumer behavior and sales activity for Store 1, providing valuable insights for strategic decision-making in retail management

In [ ]:
import matplotlib.pyplot as plt

# Store 3 with non-Holiday's
non_holiday_2 = df[(df['Store'] == 2) & (df['Holiday_Flag'] == 0)].iloc[:140]

# Store 4 with holidays
holiday_2 = df[(df['Store'] == 2) & (df['Holiday_Flag'] == 1)].iloc[:140]


# Plot sales data for Store 1 during non-holiday weeks
plt.plot(non_holiday_2.index, non_holiday_2['Weekly_Sales'], color='blue', label='Non-Holiday Weeks')

# Plot sales data for Store 1 during holiday weeks
plt.plot(holiday_2.index, holiday_2['Weekly_Sales'], color='red', label='Holiday Weeks')

plt.xlabel('Week')
plt.ylabel('Weekly Sales (In Millions) ')
plt.title('Weekly Sales for Store 2')
plt.legend()
plt.grid(True)  
plt.tight_layout()
plt.show()

In the second store, we notice that there is a significant difference in the total sales amount compared to store 1, which is much higher.

In [ ]:
import matplotlib.pyplot as plt

# Store 3 with non-Holiday's
non_holiday_3 = df[(df['Store'] == 3) & (df['Holiday_Flag'] == 0)].iloc[:140]

# Store 4 with holidays
holiday_3 = df[(df['Store'] == 3) & (df['Holiday_Flag'] == 1)].iloc[:140]


# Plot sales data for Store 1 during non-holiday weeks
plt.plot(non_holiday_3.index, non_holiday_3['Weekly_Sales'], color='blue', label='Non-Holiday Weeks')

# Plot sales data for Store 1 during holiday weeks
plt.plot(holiday_3.index, holiday_3['Weekly_Sales'], color='red', label='Holiday Weeks')

plt.xlabel('Week')
plt.ylabel('Weekly Sales')
plt.title('Weekly Sales for Store 3')
plt.legend()
plt.grid(True)  
plt.tight_layout()
plt.show()

Store 3 consistently reports among the lowest weekly sales figures compared to other stores. Despite occasional spikes during holidays, its sales performance falls significantly short of Stores 1 and 2, which achieve over 3 million and approximately 550k in total sales, respectively.

In [ ]:
import matplotlib.pyplot as plt

# Store 4 with non-Holidays
non_holiday_4 = df[(df['Store'] == 4) & (df['Holiday_Flag'] == 0)].iloc[:140]

# Store 4 with Holidays
holiday__4 = df[(df['Store'] == 4) & (df['Holiday_Flag'] == 1)].iloc[:140]

plt.plot(non_holiday_4.index, non_holiday_4['Weekly_Sales'], color='blue', label='Non-Holiday Weeks')

plt.plot(holiday__4.index, holiday__4['Weekly_Sales'], color='red', label='Holiday Weeks')

plt.xlabel('Week')
plt.ylabel('Weekly Sales(In Millions)')
plt.title('Weekly Sales for Store 4')
plt.legend()
plt.grid(True)  
plt.tight_layout()
plt.show()

Store 4 demonstrates consistent sales performance akin to that of Store 1 and Store 2. This observation suggests that Stores 1, 2, and 4 likely cater to a comparable population size and exhibit similar consumer foot traffic patterns, in contrast to Store 3.

Thus, from our analysis of stores 1 through 4, there is a similar spike in sales during holiday weeks, as shown from the visualization. This indicates that these stores will have higher revenue and a greater number of consumers visiting during these holiday weeks. Therefore, during these holiday weeks, these stores should continue to prioritize them since this is when they have the greatest revenue.

Moreover, to confirm this, we can create a hypothesis test.

*Null Hypothesis: Holiday weeks don't have an effect on weekly sales.*

*Alternative Hypothesis: Holiday weeks do have an effect on weekly sales.*

Assume our alpha level is 0.05.

We can use a t test to determine if the means of sales during holiday weeks vs. non-holiday weeks are different:

In [ ]:
holiday_diff = df.groupby("Holiday_Flag").mean("Weekly_Sales")

fig, ax = plt.subplots()

divisions = ["Non-Holiday", "Holiday"]
counts = [holiday_diff["Weekly_Sales"][0], holiday_diff["Weekly_Sales"][1]]

ax.bar(divisions, counts, color=["tab:red", "tab:orange"])

ax.set_ylabel("Weekly Sales (in million USD)")
ax.set_title("Non-Holiday & Holiday vs. Weekly Sales")

plt.show()

Now, let's run the t test to see if the differences in results are statistically significant:

In [ ]:
holiday = df[df["Holiday_Flag"] == 1]["Weekly_Sales"]
no_holiday = df[df["Holiday_Flag"] == 0]["Weekly_Sales"]

result = sci.stats.ttest_ind(holiday, no_holiday)

print(result)

Notice that our p value is 0.003, which is way lower than the alpha level of 0.05. Through this, we can reject the null hypothesis, meaning that holiday weeks do have an effect on weekly sales! We can also see the differences in the weekly sales for holidays vs. non-holiday weeks in the bar graph above.

#### Month vs. Sales

We can use a hypothesis test to figure out if the month affects the weekly sales *per store*. Note how it wouldn't make sense if we combined stores' weekly sales for each month, because some stores do way better in general than others. If we did do that, then we wouldn't be able to see the differences in month for a particular store.

*Null Hypothesis: The month of the year does not have an effect on weekly sales.*

*Alternative Hypothesis: The month of the year does have an effect on weekly sales.*

Assume a significance level of 0.05.

First of all, let's draw a graph noting the average weekly sales for each month for Store 1:

In [ ]:
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

def determine_month(date):
    return months[date.month-1]

df["Month"] = df["Date"].apply(determine_month)

def avg(lst):
    return sum(lst) / len(lst)

fig, ax = plt.subplots()

list_of_months = [[],[],[],[],[],[],[],[],[],[],[],[]]

for index in range(len(months)):
    curr_month = []
    for ind, item in df[(df["Month"] == months[index]) & (df["Store"] == 1)].iterrows():
        curr_month.append(float(item["Weekly_Sales"]))
    list_of_months[index] = avg(curr_month)

divisions = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
counts = list_of_months

ax.bar(divisions, counts, color=["tab:red"])

ax.set_ylabel("Weekly Sales (in million USD)")
ax.set_title("Month vs. Weekly Sales for Store 1")

plt.show()

Note that Store 1 seems to be doing way better in sales during the months of November, December, and February. For every other month, it seems to be doing roughly the same numbers (around 1.5 million USD)

Now, let's conduct ANOVA tests for every store.

Note that we want to see if there exists a difference in the means of weekly sales for each month *for each store*:

In [ ]:
p_value_results = []

def anova_test(store):

    january = []
    february = []
    march = []
    april = []
    may = []
    june = []
    july = []
    august = []
    september = []
    october = []
    november = []
    december = []

    list_of_months = [january, february, march, april, may, june, july, august, september, october, november, december]

    for index in range(len(months)):
        for ind, item in df[(df["Month"] == months[index]) & (df["Store"] == store)].iterrows():
            list_of_months[index].append(float(item["Weekly_Sales"]))

    result = sci.stats.f_oneway(january, february, march, april, may, june, july, august, september, october, november, december)
    
    print("Store " + str(store) + " test pvalue: " + str(result.pvalue))
    p_value_results.append(result.pvalue)

for num in range(1, 46):
    anova_test(num)

Some stores (Store 42, 43, 44, etc...) exibit higher p values upon inspection, going up to 0.66, which would allow us to fail to reject the null hypothesis. However, for the most part, most stores exibit very low p values, way lower than our significance level. This would mean that we'd have to reject the null hypothesis. Let's see how many stores had a result lower than 0.05, and those who had a result higher:

In [ ]:
num_less = 0
num_more = 0

for value in p_value_results:
    if value <= 0.05:
        num_less = num_less + 1
    elif value > 0.05:
        num_more = num_more + 1

print("Number of stores with p values less than 0.05: " + str(num_less))
print("Number of stores with p values more than 0.05: " + str(num_more))

We can conclude that for the most part, the months of the year do affect the average sales per store.